<a href="https://colab.research.google.com/github/mcollmeyer8/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [917 kB]
Hit:13 http://ppa.laun

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-21 18:12:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-02-21 18:12:50 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [38]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType, StructType, StructField

# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [7]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|            0|          0|   N|                Y|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|                Y|
| R8TVYIJXLYJT0|          5|            0|          0|   N|                Y|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|     

In [27]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter(vine_df.total_votes >= "20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

In [50]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where 
# the number of helpful_votes divided by total_votes is equal to or greater than 50%.
# Step 1: Add a column for helpful_votes % of total_votes
helpful_votes_df = total_votes_df.withColumn('helpful_votes_percentage', total_votes_df["helpful_votes"]/total_votes_df["total_votes"]*100)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|       78.26086956521739|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|       83.33333333333334|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|       37.03703703703704|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|       95.83333333333334|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|                    90.0|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|       97.36842105263158|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|              

In [51]:
# Format helpful_votes_percentage column
helpful_vote_df = helpful_votes_df.withColumn("helpful_votes_percentage", helpful_votes_df["helpful_votes_percentage"].cast(DecimalType(10,2)))
helpful_vote_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|                   78.26|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|                   83.33|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|                   37.04|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|                   95.83|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|                   90.00|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|                   97.37|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|              

In [52]:
# Step 2: Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
higher_helpful_votes_df = helpful_vote_df.filter(helpful_vote_df.helpful_votes_percentage >= "50")
higher_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|                   78.26|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|                   83.33|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|                   95.83|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|                   90.00|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|                   97.37|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|                   97.18|
|R17ARFSDV555EW|          1|           29|         29|   N|              

In [54]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where 
# a review was written as part of the Vine program (paid), vine == 'Y'.
vine_reviews_df = higher_helpful_votes_df.filter(higher_helpful_votes_df.vine == "Y")
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|                   96.40|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N|                   97.30|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|                   74.24|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|                   93.55|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|                   95.54|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|                   73.08|
|R3NTGOCUZB33JK|          5|           40|         47|   Y|              

In [55]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
no_vine_reviews_df = higher_helpful_votes_df.filter(higher_helpful_votes_df.vine == "N")
no_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|                   78.26|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|                   83.33|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|                   95.83|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|                   90.00|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|                   97.37|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|                   97.18|
|R17ARFSDV555EW|          1|           29|         29|   N|              

In [92]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
# Step 1: Determine the total number of reviews
df.count()

5115666

In [91]:
# Step 2: Determine the number of 5-star reviews
five_star_reviews_df = df.filter(df.star_rating == "5")
five_star_reviews_df.count()

3254753

In [93]:
# Step 3: Count of 5-star reviews for paid reviews
paid_five_star_reviews_df = five_star_reviews_df.filter(five_star_reviews_df.vine == "Y")
paid_five_star_reviews_df.count()

12429

In [103]:
# Step 4: Percentage of 5-star reviews for paid reviews
paid_five_star_review_percentage = (paid_five_star_reviews_df.count()/ five_star_reviews_df.count())*100
paid_five_star_review_percentage

0.38187229568572484

In [99]:
# Step 5: Count of 5-star reviews for non-paid reviews
non_paid_five_star_reviews_df = five_star_reviews_df.filter(five_star_reviews_df.vine == "N")
non_paid_five_star_reviews_df.count()

3242324

In [104]:
# Step 6: Percentage of 5-star reviews for non-paid reviews
non_paid_five_star_review_percentage = (non_paid_five_star_reviews_df.count()/ five_star_reviews_df.count())*100
non_paid_five_star_review_percentage

99.61812770431428

In [105]:
# Total Vine reviews
vine_reviews_df = df.filter(df.vine == "Y")
vine_reviews_df.count()

33309

In [106]:
# Total non-Vine reviews
vine_reviews_df = df.filter(df.vine == "N")
vine_reviews_df.count()

5082143